In [1]:
import nltk

In [ ]:
def gender_features(word):
    return {'last_letter': word[-1]}
gender_features('Shrek')

In [ ]:
# def gender_features(word):
#     features = {'last_letter': word[-1],
#                 'name_length': len(word),
#                 'first_letter': word[0]
#                }
#     return features
# gender_features('Shrek')

In [ ]:
from nltk.corpus import names

In [ ]:
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
[(name, 'female') for name in names.words('female.txt')])
type(labeled_names)

In [ ]:
import random

In [ ]:
random.shuffle(labeled_names)

In [ ]:
labeled_names

In [ ]:
len(labeled_names)

In [ ]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]

In [ ]:
train_set, test_set = featuresets[500:], featuresets[:500]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.classify(gender_features('Neo'))

In [ ]:
classifier.classify(gender_features('Trinity'))

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
classifier.show_most_informative_features(5)

In [ ]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

In [ ]:
test_set

In [ ]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrtuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [ ]:
gender_features2('John')

In [ ]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [ ]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

In [ ]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [ ]:
for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

In [ ]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

In [ ]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

In [ ]:
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [ ]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

In [ ]:
doc = movie_reviews.words('pos/cv957_8737.txt')
set(doc)

In [ ]:
featuresets = [(document_features(d), c) for (d, c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
classifier.show_most_informative_features(5)

In [ ]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [ ]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

In [ ]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [ ]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n, g) in tagged_words]

In [ ]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [ ]:
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

In [ ]:
classifier.classify(pos_features('cats'))

In [ ]:
print(classifier.pseudocode(depth=4))

In [ ]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
               "suffix(2)": sentence[i][-2:],
               "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

In [ ]:
pos_features(brown.sents()[0], 8)

In [ ]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append( (pos_features(untagged_sent, i), tag) )

In [ ]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
nltk.classify.accuracy(classifier, test_set)

In [ ]:
from nltk.corpus import brown

In [ ]:
def pos_features(sentence, i, history):
    features = {"suffix(1)": sentence[i][-1:],
               "suffix(2)": sentence[i][-2:],
               "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features

class ConsecutivePosTagger(nltk.TaggerI):
    
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in tagged_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [ ]:
nltk.TaggerI?

In [ ]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))

In [2]:
sents = nltk.corpus.treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)

In [3]:
boundaries

{1,
 90116,
 16389,
 40968,
 81929,
 24587,
 16396,
 65548,
 73741,
 8207,
 32784,
 81931,
 90128,
 98315,
 20,
 65557,
 57366,
 8221,
 24611,
 36,
 32804,
 38,
 57382,
 81957,
 98345,
 73771,
 8236,
 41004,
 8238,
 16430,
 49201,
 73785,
 49210,
 16445,
 57405,
 64,
 90176,
 66,
 32835,
 73794,
 24649,
 16459,
 65615,
 98383,
 82001,
 57426,
 49235,
 8276,
 82003,
 90195,
 32855,
 90197,
 41050,
 8285,
 73824,
 32868,
 49252,
 102,
 16487,
 65640,
 98410,
 82029,
 24688,
 49265,
 73840,
 41075,
 90227,
 16502,
 32887,
 65662,
 8319,
 24704,
 57474,
 134,
 41097,
 41099,
 49292,
 73867,
 32911,
 65686,
 57495,
 73878,
 98454,
 8346,
 82074,
 8348,
 24732,
 90267,
 32930,
 163,
 8355,
 24740,
 41126,
 57511,
 49320,
 73897,
 16554,
 82088,
 98475,
 65712,
 41137,
 57522,
 16568,
 90304,
 98497,
 82114,
 8390,
 199,
 24775,
 49352,
 57545,
 16587,
 73929,
 211,
 24788,
 65749,
 32982,
 90324,
 41180,
 57569,
 49378,
 73955,
 228,
 49380,
 98537,
 65770,
 237,
 8430,
 16622,
 33010,
 4120

In [4]:
tokens

['.',
 'START',
 'Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 ',',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov',
 '.',
 '29',
 '.',
 'Mr',
 '.',
 'Vinken',
 'is',
 'chairman',
 'of',
 'Elsevier',
 'N',
 '.',
 'V',
 '.,',
 'the',
 'Dutch',
 'publishing',
 'group',
 '.',
 '.',
 'START',
 'Rudolph',
 'Agnew',
 ',',
 '55',
 'years',
 'old',
 'and',
 'former',
 'chairman',
 'of',
 'Consolidated',
 'Gold',
 'Fields',
 'PLC',
 ',',
 'was',
 'named',
 'a',
 'nonexecutive',
 'director',
 'of',
 'this',
 'British',
 'industrial',
 'conglomerate',
 '.',
 '.',
 'START',
 'A',
 'form',
 'of',
 'asbestos',
 'once',
 'used',
 'to',
 'make',
 'Kent',
 'cigarette',
 'filters',
 'has',
 'caused',
 'a',
 'high',
 'percentage',
 'of',
 'cancer',
 'deaths',
 'among',
 'a',
 'group',
 'of',
 'workers',
 'exposed',
 'to',
 'it',
 'more',
 'than',
 '30',
 'years',
 'ago',
 ',',
 'researchers',
 'reported',
 '.',
 'The',
 'asbestos',
 'fiber',
 ',',
 'crocidolit

In [5]:
def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i+1][0].isupper(),
           'prev-word': tokens[i-1].lower(),
           'punct': tokens[i],
           'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [6]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
              for i in range(1, len(tokens)-1)
              if tokens[i] in '.?!']

In [8]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.936026936026936

In [9]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i + 1
    if start < len(words):
        sents.append(words[start:])
    return sents